In [1]:
import os
import json
import censusdata
import subprocess
import pandas as pd
from uszipcode import SearchEngine

# This is the location of my Mapshaper binary
MAPSHAPER = os.path.join(os.path.expanduser('~'), "node_modules/mapshaper/bin/mapshaper")
DATA_DIRECTORY = os.path.join(os.getcwd(), "data")

pd.set_option('display.max_rows', 1000)

### (1) Mapping Zip Codes to State FIPS Codes
Source: https://www2.census.gov/geo/docs/maps-data/data/rel/zcta_place_rel_10.txt


In [2]:
zipcode_search = SearchEngine(simple_or_comprehensive=SearchEngine.SimpleOrComprehensiveArgEnum.simple)
def get_state_abbreviation_from_zcta(zcta, search_engine=zipcode_search):
    zipcode_data = search_engine.by_zipcode(zcta)
    if zipcode_data:
        return zipcode_data.state
    else:
        return None

### (2) Mapping U.S. State Names, U.S. State Abbreviations and FIPS Codes
Source: https://gist.github.com/rogerallen/1583593

In [3]:
with open(os.path.join(DATA_DIRECTORY, "us_state_to_abbrev.json")) as f:
    us_state_to_abbrev = json.loads(f.read())

abbrev_to_us_state = dict(map(reversed, us_state_to_abbrev.items()))
    
with open(os.path.join(DATA_DIRECTORY, "us_state_abbrev_to_fips.json")) as f:
    us_state_abbrev_to_fips = json.loads(f.read())

fips_to_us_state_abbrev = dict(map(reversed, us_state_abbrev_to_fips.items()))

### (3) Read & Format Diabetic Amputation Data by State

The State Media Co. filed open records requests to gain hospital discharge data from southeastern states from 2016 to 2020. The data consisted of 76 procedural codes for nontraumatic lower extremity amputations for diabetic patients, broken down by each patient’s ZIP code of residence. Four southeastern states (Louisiana, Arkansas, North Carolina and Tennessee) either did not have data or would not provide it. ZIP code populations, according to 5-year averages from the U.S. Census Bureau, were then divided by the number of amputations for each ZIP code for each year to create a rate. For privacy reasons, some states suppressed amputation data for certain ZIP codes when there were fewer than five procedures. The newspaper then took the population size of 29203 and compared it with ZIP codes the same size or larger in South Carolina, Georgia, Alabama and Mississippi.

In [4]:
def add_empty_rows_to_df(df, years=[2016, 2017, 2018, 2019, 2020]):
    zipcodes_vec = []
    years_vec = []
    amputations_vec = []

    for zipcode in df.ZIP.unique():
        df_years = df[df.ZIP == zipcode].YEAR
        empty_years = list(set(years) - set(df_years))

        for year in empty_years:
            zipcodes_vec.append(zipcode)
            years_vec.append(year)
            amputations_vec.append(0)
    
    empty_df = pd.DataFrame({
        'ZIP': zipcodes_vec,
        'YEAR': years_vec,
        'AMPUTATIONS': amputations_vec
    })
    
    return pd.concat([df, empty_df])

def format_amputations_df(df, state_fips, us_state_abbrev_to_fips):
    # Alabama's amputations column neeeds to be summed by zipcode
    # We will aggregate these rows into unique zipcode rows
    df['AMPUTATIONS'] = [value if value != "*" else 0 for value in df['AMPUTATIONS']]
    df = df.groupby(['ZIP', 'YEAR'])['AMPUTATIONS'].sum().reset_index()

    # Add empty years (i.e. 0 amputations)
    df = add_empty_rows_to_df(df)
    
    # Each file contains patient zip codes outside of their own state
    # For this investigation, we will be dropping these items from each df
    zipcodes = df['ZIP']
    state_abbreviations = [get_state_abbreviation_from_zcta(zipcode) for zipcode in zipcodes]
    df['State'] = state_abbreviations
    df['State FIPS'] = [us_state_abbrev_to_fips[abbrev] if abbrev in us_state_abbrev_to_fips else None for abbrev in state_abbreviations]
    df = df[df['State FIPS'] == state_fips].reset_index(drop=True)
    
    return df

state_abbreviations = ['AL', 'FL', 'GA', 'KY', 'MS', 'SC', 'VA', 'WV']
state_fips_codes = [us_state_abbrev_to_fips[abbrev] for abbrev in state_abbreviations]
amputations_dfs = []

for state_abbreviation in state_abbreviations:
    filename = "Diabetic Amputations Raw Data - {0}.csv".format(state_abbreviation)
    filepath = os.path.join(DATA_DIRECTORY, filename)
    df = pd.read_csv(filepath, dtype={'ZIP': str})
    state_fips = us_state_abbrev_to_fips[state_abbreviation]
    formatted_df = format_amputations_df(df, state_fips, us_state_abbrev_to_fips)
    amputations_dfs.append(formatted_df)
    
amputations_df = pd.concat(amputations_dfs).sort_values(by=['State', 'ZIP', 'YEAR'])

In [5]:
amputations_df[amputations_df.ZIP=='29203']

,ZIP,YEAR,AMPUTATIONS,State,State FIPS
207,29203,2016,46,SC,45
208,29203,2017,49,SC,45
209,29203,2018,35,SC,45
210,29203,2019,38,SC,45
211,29203,2020,31,SC,45


### (4) Get Population Data by Year
The Census data we used comes from the ACS 5-year estimates at the ZCTA level (approximate polygons for ZIP codes, which are really delivery routes). We used the 5-year averages since a ZIP code is a very small geographic unit for ACS survey methods, and there simply isn’t high enough precision to get reliable estimates from the 1-year ACS or 3-year ACS datasets.

In [6]:
# https://api.census.gov/data/2010/acs/acs5/variables.html
# B01003_001E - Estimate!!Total - TOTAL POPULATION
# API - https://api.census.gov/data/2016/acs/acs5?get=NAME,ZCTA,B01003_001E&for=zip+code+tabulation+area:*&in=state:12
years = [2016, 2017, 2018, 2019, 2020]
census_product = 'acs5'
fips_string = ','.join(state_fips_codes)
code_string = 'B01003_001E'
year_zcta_population_dict = {}

for year in years:
    # API didn't seem to support the "in" query for specifying states in 2020
    if year == 2020:
        population_table = censusdata.download(census_product, year,
           censusdata.censusgeo([
               ('zip%20code%20tabulation%20area', '*')
           ]),['ZCTA',code_string])
    else:
        population_table = censusdata.download(census_product, year,
           censusdata.censusgeo([
               ('state', fips_string),
               ('zip%20code%20tabulation%20area', '*')
           ]),['ZCTA',code_string])
    zcta_population_dict = dict(zip(population_table.ZCTA, population_table.B01003_001E))
    year_zcta_population_dict[year] = zcta_population_dict
    print("Finished processing {0} {1}".format(census_product, year))

Finished processing acs5 2016
Finished processing acs5 2017
Finished processing acs5 2018
Finished processing acs5 2019
Finished processing acs5 2020


In [7]:
# Get population from the back up 2010 Census
# There were many ZIP codes with no population, so this was a double check
# to see if that data were erroneous or retrievable (albeit through an old decennial)
census_2010_population = censusdata.download('sf1', 2010,
       censusdata.censusgeo([
           ('state', fips_string),
           ('zip code tabulation area (or part)', '*')
       ]),['ZCTA5','P001001'])
census_2010_zcta_population_dict = dict(zip(census_2010_population.ZCTA5, census_2010_population.P001001))    

### (5) Add Population Data to Amputations Data
Similarly, we used 5-year averages as more reliable estimates for the amputation rate of a given ZIP code. We also calculated these rates per 10K residents and per 40K residents.

In [8]:
population_vec = []

for i, row in amputations_df.iterrows():
    zcta = int(row.ZIP)
    year = row.YEAR
    amputations = row.AMPUTATIONS
    if zcta not in year_zcta_population_dict[year]:
        in_2010_census = zcta in census_2010_zcta_population_dict
#         print("WARNING: No population found for {0} in {1} {2}".format(zcta, year, in_2010_census))
        if in_2010_census:
            print("FOUND {0} population in Decennial Census of 2010".format(zcta))
            population = census_2010_zcta_population_dict[zcta]
        else:
            population = 0
    else:
        population = year_zcta_population_dict[year][zcta]
    population_vec.append(population)

amputations_df['POPULATION'] = population_vec
amputations_df['AMPUTATIONS_PER_10K'] = [10000 * float(amputations_df.AMPUTATIONS.iloc[i]) / float(amputations_df.POPULATION.iloc[i]) if amputations_df.POPULATION.iloc[i] != 0 else None  for i in range(len(amputations_df))]
amputations_df['AMPUTATIONS_PER_40K'] = [40000 * float(amputations_df.AMPUTATIONS.iloc[i]) / float(amputations_df.POPULATION.iloc[i]) if amputations_df.POPULATION.iloc[i] != 0 else None  for i in range(len(amputations_df))]

avg_populations_dict = amputations_df.groupby(["ZIP"])['POPULATION'].mean().to_dict()
amputations_df['AVERAGE POPULATION (2016-2020)'] = [avg_populations_dict[z] for z in amputations_df.ZIP]


In [9]:
amputations_df

,ZIP,YEAR,AMPUTATIONS,State,State FIPS,POPULATION,AMPUTATIONS_PER_10K,AMPUTATIONS_PER_40K,AVERAGE POPULATION (2016-2020)
1763,35004,2016,0,AL,01,10418,0.000000,0.000000,11467.2
0,35004,2017,3,AL,01,11167,2.686487,10.745948,11467.2
1,35004,2018,3,AL,01,11762,2.550587,10.202347,11467.2
1764,35004,2019,0,AL,01,12045,0.000000,0.000000,11467.2
2,35004,2020,2,AL,01,11944,1.674481,6.697924,11467.2
...,...,...,...,...,...,...,...,...,...
1552,26852,2016,0,WV,54,782,0.000000,0.000000,814.6
2727,26852,2017,0,WV,54,897,0.000000,0.000000,814.6
2728,26852,2018,0,WV,54,809,0.000000,0.000000,814.6
1553,26852,2019,0,WV,54,742,0.000000,0.000000,814.6


### (6) Get Insurance Data by Year
```
S2701_C01_001E - Total!!Estimate!!Civilian noninstitutionalized population
S2701_C04_001E - Uninsured!!Estimate!!Civilian noninstitutionalized population
S2701_C05_001E - Percent Uninsured!!Estimate!!Civilian noninstitutionalized population
```

In [10]:
# https://api.census.gov/data/2015/acs/acs5/subject/groups/S2701.html
# S2701_C01_001E - Total!!Estimate!!Civilian noninstitutionalized population
# S2701_C04_001E - Uninsured!!Estimate!!Civilian noninstitutionalized population
# S2701_C05_001E - Percent Uninsured!!Estimate!!Civilian noninstitutionalized population
# https://api.census.gov/data/2015/acs/acs5/subject/?get=NAME,ZCTA,S2701_C05_001E&for=zip+code+tabulation+area:*&in=state:12


year_zcta_insurance_dict = {}

for year in years:
    insurance_table = censusdata.download('acs5/subject', year,
           censusdata.censusgeo([
               ('zip code tabulation area', '*')
           ]),['ZCTA','S2701_C04_001E', 'S2701_C01_001E', 'S2701_C05_001E'])
    zcta_insurance_dict = {}
    for i, zcta in enumerate(list(insurance_table.ZCTA)):
        zcta_insurance_dict[zcta] = {
            "NONINSTITUTIONALIZED POPULATION": insurance_table.S2701_C01_001E.iloc[i],
            "UNINSURED POPULATION (NON-I)": insurance_table.S2701_C04_001E.iloc[i],
            "UNINSURED PERCENTAGE (NON-I)": insurance_table.S2701_C05_001E.iloc[i]
        }
    year_zcta_insurance_dict[year] = zcta_insurance_dict
    print("Finished processing {0} {1}".format('acs5/subject', year))


Finished processing acs5/subject 2016
Finished processing acs5/subject 2017
Finished processing acs5/subject 2018
Finished processing acs5/subject 2019
Finished processing acs5/subject 2020


### (7) Add Insurance Data to Amputations Data
Merged insurance data with amputation data.

In [11]:
noninstitutionalized_vec = []
uninsured_pop_vec = []
uninsured_percentage_vec = []

for i, row in amputations_df.iterrows():
    zcta = int(row.ZIP)
    year = row.YEAR
    if zcta not in year_zcta_insurance_dict[year]:
        noninstitutionalized_vec.append(None)
        uninsured_pop_vec.append(None)
        uninsured_percentage_vec.append(None)
    elif year_zcta_insurance_dict[year][zcta]['UNINSURED PERCENTAGE (NON-I)'] == -666666666:
        noninstitutionalized_vec.append(None)
        uninsured_pop_vec.append(None)
        uninsured_percentage_vec.append(None)
    else: 
        noninstitutionalized_vec.append(year_zcta_insurance_dict[year][zcta]['NONINSTITUTIONALIZED POPULATION'])
        uninsured_pop_vec.append(year_zcta_insurance_dict[year][zcta]['UNINSURED POPULATION (NON-I)'])
        uninsured_percentage_vec.append(year_zcta_insurance_dict[year][zcta]['UNINSURED PERCENTAGE (NON-I)'])

    
amputations_df['NONINSTITUTIONALIZED POPULATION'] = noninstitutionalized_vec
amputations_df['UNINSURED POPULATION (NON-I)'] = uninsured_pop_vec
amputations_df['UNINSURED PERCENTAGE (NON-I)'] = uninsured_percentage_vec


In [12]:
amputations_df

,ZIP,YEAR,AMPUTATIONS,State,State FIPS,POPULATION,AMPUTATIONS_PER_10K,AMPUTATIONS_PER_40K,AVERAGE POPULATION (2016-2020),NONINSTITUTIONALIZED POPULATION,UNINSURED POPULATION (NON-I),UNINSURED PERCENTAGE (NON-I)
1763,35004,2016,0,AL,01,10418,0.000000,0.000000,11467.2,10400.0,344.0,3.3
0,35004,2017,3,AL,01,11167,2.686487,10.745948,11467.2,11146.0,559.0,5.0
1,35004,2018,3,AL,01,11762,2.550587,10.202347,11467.2,11742.0,733.0,6.2
1764,35004,2019,0,AL,01,12045,0.000000,0.000000,11467.2,12019.0,830.0,6.9
2,35004,2020,2,AL,01,11944,1.674481,6.697924,11467.2,11912.0,933.0,7.8
...,...,...,...,...,...,...,...,...,...,...,...,...
1552,26852,2016,0,WV,54,782,0.000000,0.000000,814.6,782.0,13.0,1.7
2727,26852,2017,0,WV,54,897,0.000000,0.000000,814.6,897.0,83.0,9.3
2728,26852,2018,0,WV,54,809,0.000000,0.000000,814.6,809.0,66.0,8.2
1553,26852,2019,0,WV,54,742,0.000000,0.000000,814.6,742.0,91.0,12.3


### (8) Extrapolating VA's Q1-Q3 2020 Data
Virginia's dataset does not include 2016 and only includes Q1-Q3 (inclusive) in 2020. We don't have access to the monthly/daily distribution of amputations, so here we are extrapolating linearly to impute values for Q4, then rounding to whole numbers for the final extrapolated total for VA 2020 amputation data.

In [13]:
amputations_df.loc[
    amputations_df.State == 'VA', 'AMPUTATIONS'
] = amputations_df.loc[amputations_df.State == 'VA', 'AMPUTATIONS'].apply(lambda x: round(int(x)/.75))

In [14]:
amputations_df = amputations_df.drop(['State FIPS', 'UNINSURED PERCENTAGE (NON-I)'], 1)

/var/folders/1m/gs67qnm13_zdcfgmdqmjy_sh0000gp/T/ipykernel_54145/1500814753.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  amputations_df = amputations_df.drop(['State FIPS', 'UNINSURED PERCENTAGE (NON-I)'], 1)


In [15]:
amputations_df.to_csv(os.path.join(DATA_DIRECTORY, 'amputations.csv'),index=False)

### (9) Calculating Averages and Rankings

This output looks at ZIP codes above 39000 people across Mississippi, Alabama, Georgia and South Carolina from 2016-2020. The purpose is to find rankings and average amputation rates across these years.

#### Methodology
The authors' chosen methodology was to compare population-wide diabetic amputation rates for ZIP codes **the same size or larger** than the point of reference of 29203 (whose 5-year average population is ~39.9K). However, there are many other options for methods of comparison. 

From a data similarity perspective, it is likely more appropriate to compare 29203 to a range of populations, both above and below its own population of 39K: for instance a range between 20K and 60K. If we followed this methodology, 29203's amputation rate would have the 11th-worst amputation rate instead of 1st-worst over the 5-year span (when including MS, AL, GA and SC). 

If we also add the states of Kentucky, West Virginia and Virginia (whose data was also collected by The State Media Co. - Florida was also collected, though there's a chance the methodology may count individual amputation procedures rather than number of individuals who received amputations) into this methodology, 29203 would rank 34th-worst. Separately, it may make sense to compare ZIP codes with similar population densities amongst ZIP codes instead. 

From a data metric perspective, a population-wide diabetic amputation rate is not the best metric for care specific to people with diabetes. If 100% of the population of a 10,000 person town had diabetes, and 10 people recieved amputations, the population-wide amputation rate would be **10 per 10K residents**. The diabetic-population amputation rate would be **10 per 10K diabetics**, since everyone has diabetes.

Now, if 10% of the population of 10,000 person town had diabetes (i.e. 1000 people), and 10 people still received amputations, the population-wide amputation rate would still be **10 per 10K residents**. However, the diabetic-population amputation rate would now be **100 per 10K diabetics%** (5/10). The second group of diabetics are receiving significantly worse care, with much worse outcomes. This metric would better encapsulate a comparison of diabetic amputations across ZIP codes as well. 

In [16]:
amputations_df['AMPUTATIONS'] = amputations_df['AMPUTATIONS'].astype(int)
population_filter = 39000
state_filter = ['MS', 'AL', 'GA', 'SC']

In [17]:
filtered_df = amputations_df[
    (amputations_df['AVERAGE POPULATION (2016-2020)'] > population_filter) &
    (amputations_df['State'].isin(state_filter))
]
print("Num filtered ZIP codes: {0}".format(len(filtered_df.ZIP.unique())))

Num filtered ZIP codes: 120


In [18]:
years = [2016, 2017, 2018, 2019, 2020]

filtered_cols = ['ZIP', 'State', 'AVERAGE POPULATION (2016-2020)']
filtered_zipcodes_states = filtered_df[filtered_cols].drop_duplicates().sort_values(by=['ZIP']).reset_index(drop=True)
filtered_amputations_by_year = filtered_df.pivot(index='ZIP', columns='YEAR', values='AMPUTATIONS').reset_index()
filtered_population_by_year = filtered_df.pivot(index='ZIP', columns='YEAR', values='POPULATION').reset_index()
filtered_amputation_rate_by_year = filtered_amputations_by_year[years] / filtered_population_by_year[years] * 10000
filtered_amputation_rank_by_year = filtered_amputation_rate_by_year.rank(ascending=False).astype(int)


In [19]:
# Avg. Amputations Per 10K (2016-2020)

avg_amputations_per_10K = []

for i, row in filtered_zipcodes_states.iterrows():
    
    # We suspect Alabama's data for 2016 and 2017 is undercounted, so we are calculating
    # the state's averages only over 2018, 2019 and 2020
    if row.State == 'AL':
        col_indices = [3,4,5]
    elif row.State == 'VA':
        # Similarly Virginia's methodology is only valid from 2017-2020
        col_indices = [2,3,4,5]
    else:
        col_indices = [1,2,3,4,5]
    
    sum_amputations = filtered_amputations_by_year.iloc[i, col_indices].sum()
    sum_population = filtered_population_by_year.iloc[i, col_indices].sum()
    avg_amputations = round(sum_amputations / sum_population * 10000, 2)
    avg_amputations_per_10K.append(avg_amputations)

avg_amputations_per_10K_df = pd.DataFrame({
    'Avg. Amputations Per 10K (2016-2020)': avg_amputations_per_10K
})

In [20]:
filtered_ranks_rates_df = pd.concat([
    filtered_zipcodes_states,
    filtered_amputation_rank_by_year,
    avg_amputations_per_10K_df
], axis = 1)

In [21]:
filtered_ranks_rates_df = filtered_ranks_rates_df.sort_values(
    by=['Avg. Amputations Per 10K (2016-2020)'], ascending=[False]
)

In [22]:
def get_city_from_zipcode(x):
    city = SearchEngine().by_zipcode(x).major_city
    return city if city else 'None'

cities = [get_city_from_zipcode(zipcode) for zipcode in filtered_ranks_rates_df['ZIP']]
filtered_ranks_rates_df['City'] = cities

rankingData = []

for i, row in filtered_ranks_rates_df.iterrows():
    rankingData.append(
        {
            "zip": str(row['ZIP']),
            "city": row['City'],
            "state": row['State'],
            "state_name": abbrev_to_us_state[row['State']],
            "rank_2016": row[2016],
            "rank_2017": row[2017],
            "rank_2018": row[2018],
            "rank_2019": row[2019],
            "rank_2020": row[2020],
            "amputation_rate": row['Avg. Amputations Per 10K (2016-2020)'],
            "pop": int(row['AVERAGE POPULATION (2016-2020)'])
        }
    )

In [23]:
with open(os.path.join(DATA_DIRECTORY, 'table_rates_rankings.json'), 'w') as f:
    json.dump(rankingData, f)

### (10) Finding population densities

We need to add population densities and a number of other features into the Census dataset of zipcodes. And we need to remove other fields that will only take up space. The Census ZCTA shapefile isn't broken down by state (likely because there are many ZIP codes whose area intersects multiple states). So the first few steps are an attempt to geographically filter the zipcodes using Census state shapefiles. For these steps, you need to install [Mapshaper](https://github.com/mbloch/mapshaper)

In [24]:
# Manually download and unzip Zip Code Tabulation Area (ZCTA) shapefile from the Census into your DATA_DIRECTORY
# https://www2.census.gov/geo/tiger/TIGER2021/ZCTA520/tl_2021_us_zcta520.zip

states_census_filepath = os.path.join(DATA_DIRECTORY, 'tl_2021_us_state', 'tl_2021_us_state.shp')
deep_south_filepath = os.path.join(DATA_DIRECTORY, 'deep_south.json')
filter_expression = '"28,01,13,45".indexOf(STATEFP) > -1'

subprocess.run([MAPSHAPER, states_census_filepath, "-filter", filter_expression, "-o", deep_south_filepath])

[filter] Retained 4 of 56 features
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/deep_south.json


CompletedProcess(args=['/Users/davidnewcomb/node_modules/mapshaper/bin/mapshaper', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/tl_2021_us_state/tl_2021_us_state.shp', '-filter', '"28,01,13,45".indexOf(STATEFP) > -1', '-o', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/deep_south.json'], returncode=0)

In [25]:
zcta_census_filepath = os.path.join(DATA_DIRECTORY, 'tl_2021_us_zcta520', 'tl_2021_us_zcta520.shp')
zcta_deep_south_filepath = os.path.join(DATA_DIRECTORY, 'zcta_deep_south.json')

subprocess.run([MAPSHAPER, zcta_census_filepath, "-clip", deep_south_filepath, "-o", zcta_deep_south_filepath])

[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/zcta_deep_south.json


CompletedProcess(args=['/Users/davidnewcomb/node_modules/mapshaper/bin/mapshaper', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/tl_2021_us_zcta520/tl_2021_us_zcta520.shp', '-clip', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/deep_south.json', '-o', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/zcta_deep_south.json'], returncode=0)

In [26]:
zcta_filepath = os.path.join(DATA_DIRECTORY, 'zipcodes.json')

subprocess.run([MAPSHAPER, zcta_deep_south_filepath, "-simplify", "12%", "-o", "format=topojson", zcta_filepath])

[simplify] Repaired 172 intersections
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/zipcodes.json


CompletedProcess(args=['/Users/davidnewcomb/node_modules/mapshaper/bin/mapshaper', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/zcta_deep_south.json', '-simplify', '12%', '-o', 'format=topojson', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/zipcodes.json'], returncode=0)

Now that we have filtered ZCTA shapes, we can focus on getting population densities, amputation rates, and other information added into the `zipcodes.json` file.

In [27]:
deep_south_df = amputations_df[amputations_df.State.isin(['SC', 'AL', 'MS', 'GA'])].reset_index(drop=True)
deep_south_df['AMPUTATIONS'] = deep_south_df['AMPUTATIONS'].astype(int)

deep_south_avg_populations_df = deep_south_df.groupby(["ZIP"])['POPULATION'].mean()
deep_south_states_df = deep_south_df.groupby(["ZIP"])['State'].first()
deep_south_amputations_df = deep_south_df.groupby(["ZIP"])['AMPUTATIONS'].sum()
deep_south_amputation_rates_df = deep_south_amputations_df / deep_south_df.groupby(["ZIP"])['POPULATION'].sum()*10000

#### Methodology

Alabama's dataset was more accurately reported for 2018-2020, and Virginia's was more accurately reported for 2017-2020. For the overall statistics on number of amputations and amputation rate per 10K residents, these numbers are aggregated/averaged over 3 and 4 years, respectively.

In [28]:
AL_df = deep_south_df[(deep_south_df['YEAR'].isin([2018, 2019, 2020])) & (deep_south_df['State'] == 'AL')]
AL_amputations_df = AL_df.groupby(["ZIP"])['AMPUTATIONS'].sum()
AL_amputation_rates_df = AL_amputations_df / AL_df.groupby(["ZIP"])['POPULATION'].sum()*10000

VA_df = deep_south_df[(deep_south_df['YEAR'].isin([2017, 2018, 2019, 2020])) & (deep_south_df['State'] == 'VA')]
VA_amputations_df = VA_df.groupby(["ZIP"])['AMPUTATIONS'].sum()
VA_amputation_rates_df = VA_amputations_df / VA_df.groupby(["ZIP"])['POPULATION'].sum()*10000

In [29]:
with open(zcta_filepath) as f:
    zipcodes_json = json.loads(f.read())
    
final_data = {
    'ZIP': [],
    'POPDENSITY': [],
    'POPULATION_5_YR_AVG': [],
    'AMPUTATION_RATE_5_YR_AVG_PER_10K': [],
    'AMPUTATIONS': [],
    'STATE': []
}
fields_to_remove = [
    'GEOID20', 'CLASSFP20', 'MTFCC20', 'FUNCSTAT20', 'ALAND20', 'AWATER20', 'INTPTLAT20', 'INTPTLON20'
]

for z in zipcodes_json['objects']['zcta_deep_south']['geometries']:
    zipcode_code = z['properties']['ZCTA5CE20']
    
    # Get land in square miles
    zipcode_land = z['properties']['ALAND20'] * 3.86102e-7
    
    if zipcode_code in deep_south_avg_populations_df:
        zipcode_state = deep_south_states_df[zipcode_code]
        zipcode_population = deep_south_avg_populations_df[zipcode_code]
        
        if zipcode_population > 0:
            zipcode_density = round(float(zipcode_population) / float(zipcode_land) , 2)

            if zipcode_state == 'AL':
                zipcode_amputation_rate_avg = AL_amputation_rates_df[zipcode_code]
                zipcode_amputations = AL_amputations_df[zipcode_code]
            else:
                zipcode_amputation_rate_avg = deep_south_amputation_rates_df[zipcode_code]
                zipcode_amputations = deep_south_amputations_df[zipcode_code]
            
            z['properties']['POPDENSITY'] = zipcode_density
            z['properties']['POPULATION_5_YR_AVG'] = zipcode_population
            z['properties']['AMPUTATION_RATE_5_YR_AVG_PER_10K'] = round(zipcode_amputation_rate_avg, 2)
            z['properties']['AMPUTATIONS'] = zipcode_amputations.astype(str)
            z['properties']['STATE'] = zipcode_state

            final_data['ZIP'].append(zipcode_code)
            final_data['POPDENSITY'].append(zipcode_density)
            final_data['POPULATION_5_YR_AVG'].append(zipcode_population)
            final_data['AMPUTATION_RATE_5_YR_AVG_PER_10K'].append(round(zipcode_amputation_rate_avg, 2))
            final_data['AMPUTATIONS'].append(zipcode_amputations)
            final_data['STATE'].append(zipcode_state)
        else:
            z['properties']['POPDENSITY'] = 0
            z['properties']['POPULATION_5_YR_AVG'] = 0
            z['properties']['AMPUTATION_RATE_5_YR_AVG_PER_10K'] = 0
            z['properties']['AMPUTATIONS'] = 0
            z['properties']['STATE'] = deep_south_states_df[zipcode_code]
    else:
        z['properties']['POPDENSITY'] = 0
        z['properties']['POPULATION_5_YR_AVG'] = 0
        z['properties']['AMPUTATION_RATE_5_YR_AVG_PER_10K'] = 0
        z['properties']['AMPUTATIONS'] = 0
        
        if zipcode_code in deep_south_states_df:
            z['properties']['STATE'] = deep_south_states_df[zipcode_code]
        elif zipcode_code.startswith('30') or zipcode_code.startswith('31'):
            z['properties']['STATE'] = 'GA'
        elif zipcode_code.startswith('35') or zipcode_code.startswith('36'):
            z['properties']['STATE'] = 'AL'
        elif zipcode_code.startswith('38') or zipcode_code.startswith('39'):
            z['properties']['STATE'] = 'MS'
        elif zipcode_code.startswith('29') or zipcode_code.startswith('39'):
            z['properties']['STATE'] = 'SC'
        else:
            print(zipcode_code)
    
    for field in fields_to_remove:
        if field in z['properties']:
            del z['properties'][field]

final_df = pd.DataFrame(final_data)

In [30]:
with open(os.path.join(DATA_DIRECTORY, 'zipcodes_final.json'), 'w') as f:
    json.dump(zipcodes_json, f)

For mapping purposes it can help to simplify the geographies, resulting in a smaller file.

In [31]:
zipcodes_filepath = os.path.join(DATA_DIRECTORY, 'zipcodes_final.json')
zipcodes_condensed_filepath = os.path.join(DATA_DIRECTORY, 'zipcodes_condensed.json')

subprocess.run([MAPSHAPER, zipcodes_filepath, "-simplify", "70%", "-o", zipcodes_condensed_filepath])

[simplify] Repaired 166 intersections; 16 intersections could not be repaired
[o] Wrote /Users/davidnewcomb/notebooks/data/diabetic_amputations/zipcodes_condensed.json


CompletedProcess(args=['/Users/davidnewcomb/node_modules/mapshaper/bin/mapshaper', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/zipcodes_final.json', '-simplify', '70%', '-o', '/Users/davidnewcomb/notebooks/data/diabetic_amputations/zipcodes_condensed.json'], returncode=0)

### (11) Looking into Poverty Data 

Ultimately, for the article we used Low-to-Moderate Income numbers by census tract, found at this [HUD map](https://hudgis-hud.opendata.arcgis.com/datasets/HUD::low-to-moderate-income-population-by-tract/explore?filters=eyJTVEFURSI6WyI0NSJdLCJDT1VOVFkiOlsiMDc5IiwiMDYzIiwiMDE3Il19&location=34.004900%2C-81.118087%2C9.97) (whose data is downloadable). This section gathers poverty data for Lexington County (FIPS: 063) and Richland County (FIPS: 079) in South Carolina.

In [32]:
# FIPS Codes
# 063 - Lexington County, SC
# 079 - Richland County, SC

# ACS5 Variables
# B06012_002E - Population
# B06012_002E - Estimate!!Total!!Below 100 percent of the poverty level

poverty_table = censusdata.download('acs5', 2019,
   censusdata.censusgeo([
       ('state', '45'),
       ('county', '063,079'),
       ('tract', '*')
   ]),['TRACT','B01003_001E', 'B06012_001E', 'B06012_002E', 'B06012_003E', 'B06012_004E'])

In [33]:
tract_ids = [
    "000400",
    "000300",
    "000100",
    "001000",
    "010200",
    "010102",
    "011404",
    "010703",
    "010702",
    "010900",
    "010804",
    "000600",
    "000900",
    "000500",
    "010600",
    "010502",
    "010701",
    "000700",
    "010501",
    "000200",
    "010805",
    "010806"
]

poverty_rates = []
for tract_id in tract_ids:
    total_pop = poverty_table[poverty_table.TRACT == int(tract_id)]['B06012_001E']
    poverty_pop = poverty_table[poverty_table.TRACT == int(tract_id)]['B06012_002E']
    poverty_rate = round(float(int(poverty_pop)) / float(int(total_pop)), 4)
    poverty_rates.append(poverty_rate)
    
dict(zip(tract_ids, poverty_rates))
    

{'000400': 0.2268,
 '000300': 0.4683,
 '000100': 0.278,
 '001000': 0.438,
 '010200': 0.1466,
 '010102': 0.1702,
 '011404': 0.1908,
 '010703': 0.4105,
 '010702': 0.2626,
 '010900': 0.5144,
 '010804': 0.3386,
 '000600': 0.1474,
 '000900': 0.579,
 '000500': 0.4557,
 '010600': 0.3943,
 '010502': 0.488,
 '010701': 0.1924,
 '000700': 0.1924,
 '010501': 0.1599,
 '000200': 0.2671,
 '010805': 0.199,
 '010806': 0.8177}

In [34]:
poverty_table

,TRACT,B01003_001E,B06012_001E,B06012_002E,B06012_003E,B06012_004E
"Census Tract 25, Richland County, South Carolina: Summary level: 140, state:45> county:079> tract:002500",2500,3743,3695,280,135,3280
"Census Tract 26.02, Richland County, South Carolina: Summary level: 140, state:45> county:079> tract:002602",2602,3026,3011,516,248,2247
"Census Tract 104.08, Richland County, South Carolina: Summary level: 140, state:45> county:079> tract:010408",10408,4439,200,183,13,4
"Census Tract 206.05, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020605",20605,2457,2457,223,218,2016
"Census Tract 211.06, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:021106",21106,2825,2825,103,273,2449
"Census Tract 205.08, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020508",20508,2051,1959,51,75,1833
"Census Tract 203, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020300",20300,3949,3863,1139,685,2039
"Census Tract 205.11, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020511",20511,3552,3244,305,647,2292
"Census Tract 202.01, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020201",20201,2864,2864,752,373,1739
"Census Tract 206.01, Lexington County, South Carolina: Summary level: 140, state:45> county:063> tract:020601",20601,4384,4384,478,566,3340


### (12) Vehicle Ownership Data

A crucial factor of the USDA's Low-Income, Low-Access designation (modern, more descriptive version of the colloquial food desert) is vehicle ownership. This looks at the Census data for vehicle ownership by ZCTA.

In [38]:
# https://api.census.gov/data/2010/acs/acs5/variables.html
# B01003_001E - Estimate!!Total - TOTAL POPULATION
# API - https://api.census.gov/data/2016/acs/acs5?get=NAME,ZCTA,B01003_001E&for=zip+code+tabulation+area:*&in=state:12
years = [2016, 2017, 2018, 2019, 2020]
census_product = 'acs5'
fips_string = ','.join(state_fips_codes)
code_string = 'B08201_002E'
year_zcta_vehicle_dict = {}

for year in years:
    # API didn't seem to support the "in" query for specifying states
    if year == 2020:
        vehicle_table = censusdata.download(census_product, year,
           censusdata.censusgeo([
               ('zip%20code%20tabulation%20area', '*')
           ]),['ZCTA','B08201_001E','B08201_002E'])
    else:
        vehicle_table = censusdata.download(census_product, year,
           censusdata.censusgeo([
               ('state', fips_string),
               ('zip%20code%20tabulation%20area', '*')
           ]),['ZCTA','B08201_001E','B08201_002E'])
        
    zcta_vehicle_dict = {}
    for i, zcta in enumerate(list(vehicle_table.ZCTA)):
        zcta_vehicle_dict[zcta] = {
            "Total Households": vehicle_table.B08201_001E.iloc[i],
            "No Vehicle": vehicle_table.B08201_002E.iloc[i],
            "No Vehicle %": vehicle_table.B08201_002E.iloc[i] / vehicle_table.B08201_001E.iloc[i] * 100
        }
    year_zcta_vehicle_dict[year] = zcta_vehicle_dict
    print("Finished processing {0} {1}".format(census_product, year))

/var/folders/1m/gs67qnm13_zdcfgmdqmjy_sh0000gp/T/ipykernel_54145/2848875190.py:29: RuntimeWarning: invalid value encountered in long_scalars
  "No Vehicle %": vehicle_table.B08201_002E.iloc[i] / vehicle_table.B08201_001E.iloc[i] * 100


Finished processing acs5 2016
Finished processing acs5 2017
Finished processing acs5 2018
Finished processing acs5 2019
Finished processing acs5 2020


In [39]:
for year in years: print(year_zcta_vehicle_dict[year][29203])

{'Total Households': 14767, 'No Vehicle': 1912, 'No Vehicle %': 12.947788988961875}
{'Total Households': 14934, 'No Vehicle': 2164, 'No Vehicle %': 14.49042453461899}
{'Total Households': 15033, 'No Vehicle': 2242, 'No Vehicle %': 14.913856183063926}
{'Total Households': 15348, 'No Vehicle': 2365, 'No Vehicle %': 15.409173833724262}
{'Total Households': 15589, 'No Vehicle': 2265, 'No Vehicle %': 14.529475912502404}
